In [1]:
import numpy as np
import pandas as pd
import os
import pyodbc

from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import (
    DateRange,
    Dimension,
    Metric,
    MetricType,
    Filter,
    FilterExpression,
    FilterExpressionList,
    QuotaStatus,
    GetMetadataRequest,
    RunReportRequest,
    OrderBy
)

import openpyxl
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

import datetime
from datetime import datetime, timedelta, date

import warnings
warnings.simplefilter("ignore")

In [27]:
def sample_run_report(property_id):
    """Runs a metadata report on a Google Analytics 4 property."""

    client = BetaAnalyticsDataClient()

    request = GetMetadataRequest(name='properties/'+property_id+'/metadata')
    response = client.get_metadata(request)

    output =[]
    for dimension in response.dimensions:
        output.append({"Type": "Dimension", "API_Name": f"{dimension.api_name}", "UI_Name": f"{dimension.ui_name}", "Description": f"{dimension.description}", "Custom_definition": f"{dimension.custom_definition}", "Metric_type": "N/A"})

    for metric in response.metrics:
      output.append({"Type": "Metric", "API_Name": f"{metric.api_name}", "UI_Name": f"{metric.ui_name}", "Description": f"{metric.description}", "Custom_definition": f"{metric.custom_definition}", "Metric_type": f"{MetricType(metric.type_).name}"})
    
    df = pd.DataFrame(output)
    
    return df

def ga4_response_to_df(response):
    
    dim_len = len(response.dimension_headers)
    metric_len = len(response.metric_headers)
    all_data = []

    for row in response.rows:
        row_data = {}
        for i in range(0, dim_len):
            row_data.update({response.dimension_headers[i].name: row.dimension_values[i].value})
        for i in range(0, metric_len):
            row_data.update({response.metric_headers[i].name: row.metric_values[i].value})
        all_data.append(row_data)

    df = pd.DataFrame(all_data)
    
    return df

def get_ga4_report_df(property_id, dimensions, metrics, start_date, end_date):
    
    dimensions_ga4 = []
    for dimension in dimensions:
        dimensions_ga4.append(Dimension(name=dimension))

    metrics_ga4 = []
    for metric in metrics:
        metrics_ga4.append(Metric(name=metric))
    
    iter = 0
    offset = 0
    limit=250000

    while True:

        client = BetaAnalyticsDataClient()
        
        request = RunReportRequest(
            property = 'properties/'+property_id,
            return_property_quota = True,
            dimensions = dimensions_ga4,
            metrics = metrics_ga4,
            date_ranges = [DateRange(start_date=start_date, end_date=end_date)],
            #order_bys=[OrderBy(dimension=OrderBy.DimensionOrderBy(dimension_name="date"), desc=False)],
            dimension_filter=FilterExpression(
                and_group=FilterExpressionList(
                    expressions=[                    
                        FilterExpression(
                            filter=Filter(
                                field_name="customEvent:Codigo_consultora", 
                                string_filter=Filter.StringFilter(match_type=Filter.StringFilter.MatchType.PARTIAL_REGEXP, value="\d")
                            )
                        ),
                        FilterExpression(
                            filter=Filter(
                                field_name="customEvent:Titulo_consultora",
                                in_list_filter=Filter.InListFilter(
                                    values=[
                                        "Aspirante a Director Independiente",
                                        "Aspirante a Directora Independiente",
                                        "Consultor Emprendedor Independiente",
                                        "Consultora Emprendedora Independiente",
                                        "Consultor Emprendedor Independiente Senior",
                                        "Consultora Emprendedora Independiente Senior",
                                        "Consultor Independiente",
                                        "Consultora Independiente",
                                        "Director Independiente Junior",
                                        "Directora Independiente Junior",
                                        "Director Independiente Senior",
                                        "Directora Independiente Senior",
                                        "Director Independiente Super Senior",
                                        "Directora Independiente Super Senior",
                                        "Director Independiente Master",
                                        "Directora Independiente Master",
                                        "Director Independiente Regional",
                                        "Directora Independiente Regional",
                                        "Director Independiente Regional estrella",
                                        "Directora Independiente Regional Estrella'",
                                        "Director Independiente Elite Oro",
                                        "Directora Independiente Elite Oro",
                                        "Director Independiente Elite Platino",
                                        "Directora Independiente Elite Platino",
                                        "Directora Independiente Elite Diamante"
                                    ]
                                ),
                            )
                        )
                    ],
                )
            ),
            limit=limit,
            offset=offset
        )
        
        response = client.run_report(request)
        
        if iter == 0:
            
            df_result = ga4_response_to_df(response)
            iter = iter + 1

        elif iter > 0:

            if not response.rows:
                break
            else:
                df_temp = ga4_response_to_df(response)
                df_result = pd.concat([df_result, df_temp], ignore_index=True)
                iter = iter + 1

        offset += limit

    return df_result

def fn_preprocesamiento(df, items_list, view_pais):
    
    #   Primer Filtro: Todos los pasos de Pase de Pedidos
    str_list = '|'.join(items_list)
    df['Conditional'] = df['pagePath'].str.contains(str_list)
    df = df[df['Conditional'] == True]

    df.drop(['Conditional'], axis=1, inplace=True)
    df.reset_index(drop=True, inplace=True)

    #   Agregar columna View del pais
    df['View'] = view_pais

    #   Tratamiento de genero
    df['customEvent:Titulo_consultora'] = df['customEvent:Titulo_consultora'].str.replace('Consultor','Consultora')
    df['customEvent:Titulo_consultora'] = df['customEvent:Titulo_consultora'].str.replace('Director','Directora')
    df['customEvent:Titulo_consultora'] = df['customEvent:Titulo_consultora'].str.replace('Emprendedor','Emprendedora')
    df['customEvent:Titulo_consultora'] = df['customEvent:Titulo_consultora'].str.replace('estrella','Estrella')
    df['customEvent:Titulo_consultora'] = df['customEvent:Titulo_consultora'].str.replace('aa','a')

    #   Agregar columna Segment por tipo de Audiencia
    conditions = [
        (df['customEvent:Titulo_consultora'] == 'Aspirante a Directora Independiente'),
        
        (df['customEvent:Titulo_consultora'] == 'Consultora Emprendedora Independiente') | 
        (df['customEvent:Titulo_consultora'] == 'Consultora Emprendedora Independiente Senior'),
        
        (df['customEvent:Titulo_consultora'] == 'Consultora Independiente'),

        (df['customEvent:Titulo_consultora'] == 'Directora Independiente Junior') |
        (df['customEvent:Titulo_consultora'] == 'Directora Independiente Senior') | 
        (df['customEvent:Titulo_consultora'] == 'Directora Independiente Super Senior') | 
        (df['customEvent:Titulo_consultora'] == 'Directora Independiente Master') | 
        (df['customEvent:Titulo_consultora'] == 'Directora Independiente Regional') | 
        (df['customEvent:Titulo_consultora'] == 'Directora Independiente Regional Estrella') | 
        (df['customEvent:Titulo_consultora'] == 'Directora Independiente Elite Oro') | 
        (df['customEvent:Titulo_consultora'] == 'Directora Independiente Elite Platino') |
        (df['customEvent:Titulo_consultora'] == 'Directora Independiente Elite Diamante')
        ]
    choices = [
        'Audiencia Aspirante',
        'Audiencia CEM',
        'Audiencia Consultor',
        'Audiencia Director',
        ]
    df['Segment'] = np.select(conditions, choices, default="No definido")

    df['pagePath'] = df['pagePath'].str.replace('/j6.yanbalcolombia.com','')
    df['pagePath'] = df['pagePath'].str.replace('j6.yanbalcolombia.com','')

    df['pagePath'] = df['pagePath'].str.replace('/j6.yanbalperu.com','')
    df['pagePath'] = df['pagePath'].str.replace('j6.yanbalperu.com','')

    df['pagePath'] = df['pagePath'].str.replace('/j6.yanbalecuador.com','')
    df['pagePath'] = df['pagePath'].str.replace('j6.yanbalecuador.com','')

    df['pagePath'] = df['pagePath'].str.replace('/j6.yanbalbolivia.com','')
    df['pagePath'] = df['pagePath'].str.replace('j6.yanbalbolivia.com','')
    df['pagePath'] = df['pagePath'].str.replace('/www.yanbalbolivia.com','')
    df['pagePath'] = df['pagePath'].str.replace('www.yanbalbolivia.com','')

    df['Conditional'] = df['pagePath'].str.contains("receipt|spyglass")
    df = df[df['Conditional'] == False]
    df.drop(['Conditional'], axis=1, inplace=True)

    df = df[df['eventName'] ==  'page_view']
    df.drop(['eventName'], axis=1, inplace=True)

    df.reset_index(drop=True, inplace=True)
    '''
    df['Conditional'] = df['pagePath'].str.contains("click|products|incentives|payment|submit")
    df = df[df['Conditional'] == False]
    '''
    return df    

In [3]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"C:\DARIEL\01 - Credenciales\CREDENCIALES GOOGLE\API GA4 MAYA YANBAL-8a9c90e8802e.json"

#   DICCIONARIO

In [7]:
'''
df_ReporteDiccionario = sample_run_report(reporte_Peru)

#   Se crea un archivo Excell con la lista de todas las métricas a exportar
df_ReporteDiccionario.to_excel(f'Lista de Métricas y Dimensiones a Exportar_{str(date.today())}.xlsx', sheet_name = 'GA4_report', engine = 'xlsxwriter', index=False, header=True)

df_ReporteDiccionario.head()
'''

,Type,API_Name,UI_Name,Description,Custom_definition,Metric_type
0,Dimension,achievementId,Achievement ID,The achievement ID in a game for an event. Pop...,False,N/A
1,Dimension,adFormat,Ad format,Describes the way ads looked and where they we...,False,N/A
2,Dimension,adSourceName,Ad source,The source network that served the ad. Typical...,False,N/A
3,Dimension,adUnitName,Ad unit,The name you chose to describe this Ad unit. A...,False,N/A
4,Dimension,appVersion,App version,The app's versionName (Android) or short bundl...,False,N/A


#   DATOS DE CAMPAÑA (ACTUAL)

In [4]:
ecua_ds_srv = r'ECUSRVDW0\ECUA'
ecua_ds_db = r'DWUniqueYanbal'

ecua_cadena_coneccion = 'DRIVER={SQL Server Native Client 11.0};SERVER='+ecua_ds_srv+';DATABASE='+ecua_ds_db+';Trusted_Connection=yes'

#   CONSULTAR FECHA INICIO Y FIN DE CAMPANIA ACTUAL

ecua_conn = pyodbc.connect(ecua_cadena_coneccion)

query_infoCampania = '''
SELECT 
    intNumeroCampania, 
    'S'+CAST(smlNumeroSemanaCampania AS VARCHAR) smlNumeroSemanaCampania, 
    dtmFechaInicioSemana, 
    dtmFechaFinSemana
FROM DWUniqueYanbal.dbo.Tiempo
WHERE intNumeroCampania = (
    SELECT 
        intNumeroCampania
    FROM DWUniqueYanbal.dbo.Tiempo
    WHERE dtmFechaInicioSemana <= GETDATE() AND dtmFechaFinSemana >= GETDATE()
)
'''

df_infoCampania = pd.read_sql_query(query_infoCampania, ecua_conn)

ecua_conn.close()

df_infoCampania['intNumeroCampania'] = df_infoCampania['intNumeroCampania'].astype(str)
df_infoCampania['dtmFechaInicioSemana'] = df_infoCampania['dtmFechaInicioSemana'].dt.strftime('%Y-%m-%d')
df_infoCampania['dtmFechaFinSemana'] = df_infoCampania['dtmFechaFinSemana'].dt.strftime('%Y-%m-%d')

var_InicioCampania = df_infoCampania.loc[0, 'dtmFechaInicioSemana']
var_FinCampania = df_infoCampania.loc[3, 'dtmFechaFinSemana']
var_Campaña = df_infoCampania.loc[0, 'intNumeroCampania']
print("Campaña: " + var_Campaña)
print("Rango: " + var_InicioCampania + " - " + var_FinCampania)

Campaña: 202308
Rango: 2023-07-15 - 2023-08-11


#   REPORTES GOOGLE ANALYTICS

In [8]:
reporte_Colombia = '357514079'
reporte_Peru = '357541506'
reporte_Ecuador = '357508860'
reporte_Bolivia = '357528318'

dimensions = ['customEvent:Codigo_consultora','customEvent:Titulo_consultora','eventName','pagePath']
metrics = ['eventCount', 'sessions', 'averageSessionDuration', 'bounceRate']

df_GA4_COL = get_ga4_report_df(reporte_Colombia, dimensions, metrics, var_InicioCampania, var_FinCampania)
df_GA4_PER = get_ga4_report_df(reporte_Peru, dimensions, metrics, var_InicioCampania, var_FinCampania)
df_GA4_ECU = get_ga4_report_df(reporte_Ecuador, dimensions, metrics, var_InicioCampania, var_FinCampania)
df_GA4_BOL = get_ga4_report_df(reporte_Bolivia, dimensions, metrics, var_InicioCampania, var_FinCampania)

In [28]:
items_list = ['/portaldenegocio/salesorder/createorder', '/business/salesorder/createorder']

view_colombia = 'Produccion Colombia: Todos los datos de sitios web'
view_peru = 'Producción Perú: Todos los datos de sitios web'
view_ecuador = 'Producción Ecuador: Todos los datos de sitios web'
view_bolivia = 'Yanbal Bolivia: Todos los datos de sitios web (129675804)'

df_GA4_COL2 = fn_preprocesamiento(df_GA4_COL, items_list, view_colombia)
df_GA4_PER2 = fn_preprocesamiento(df_GA4_PER, items_list, view_peru)
df_GA4_ECU2 = fn_preprocesamiento(df_GA4_ECU, items_list, view_ecuador)
df_GA4_BOL2 = fn_preprocesamiento(df_GA4_BOL, items_list, view_bolivia)

In [31]:
df_GA4_AllUsers = pd.concat([df_GA4_COL2, df_GA4_PER2, df_GA4_ECU2, df_GA4_BOL2])
df_GA4_AllUsers['Segment'] = 'All Users'

df_GA4_TOTAL_Segment = pd.concat([df_GA4_COL2, df_GA4_PER2, df_GA4_ECU2, df_GA4_BOL2, df_GA4_AllUsers])

df_GA4_TOTAL_Segment.drop(['customEvent:Codigo_consultora', 'customEvent:Titulo_consultora'], axis=1, inplace=True)
df_GA4_TOTAL_Segment['Unique pageviews'] = 0
df_GA4_TOTAL_Segment['Bounces'] = 0
df_GA4_TOTAL_Segment['Exit rate'] = 0
df_GA4_TOTAL_Segment['Users'] = 1
df_GA4_TOTAL_Segment['Exits'] = 0

df_GA4_TOTAL_Segment.reset_index(inplace=True, drop=True)

df_GA4_TOTAL_Segment = df_GA4_TOTAL_Segment[['View', 'pagePath', 'Segment', 'eventCount', 'Unique pageviews', 'averageSessionDuration', 'sessions', 'Bounces', 'bounceRate', 'Exit rate', 'Users', 'Exits']]

df_GA4_TOTAL_Segment = df_GA4_TOTAL_Segment.rename(columns={'View': 'View',
                                'pagePath': 'Page path',                          
                                'Segment': 'Segment',
                                'eventCount': 'Pageviews',
                                'Unique pageviews': 'Unique pageviews',
                                'averageSessionDuration': 'Avg. time on page',
                                'sessions': 'Sessions',
                                'Bounces': 'Bounces',
                                'bounceRate': 'Bounce rate',
                                'Exit rate': 'Exit rate',
                                'Users': 'Users',
                                'Exits': 'Exits',

})

df_GA4_TOTAL_Segment['Pageviews'] = df_GA4_TOTAL_Segment['Pageviews'].astype(int)
df_GA4_TOTAL_Segment['Avg. time on page'] = df_GA4_TOTAL_Segment['Avg. time on page'].astype(float)
df_GA4_TOTAL_Segment['Avg. time on page'] = df_GA4_TOTAL_Segment['Avg. time on page'] / 60.0

df_GA4_TOTAL_Segment['Sessions'] = df_GA4_TOTAL_Segment['Sessions'].astype(int)
df_GA4_TOTAL_Segment['Bounce rate'] = df_GA4_TOTAL_Segment['Bounce rate'].astype(int)

df_GA4_TOTAL_PagexSegment = df_GA4_TOTAL_Segment.copy()
df_GA4_TOTAL_Segment.drop(['Page path'], axis=1, inplace=True)

df_GA4_TOTAL_Segment = df_GA4_TOTAL_Segment.groupby(['View', 'Segment']).sum().reset_index()
df_GA4_TOTAL_PagexSegment = df_GA4_TOTAL_PagexSegment.groupby(['View', 'Page path', 'Segment']).sum().reset_index()

#   CREAR ARCHIVO

In [33]:
file_name = "PasePedidos_"+var_Campaña+".xlsx"

#   Primera Hoja

df_GA4_TOTAL_Segment.to_excel(file_name, header=True, index=False, sheet_name='Segment')

#   Segunda Hoja

file_name = r"./"+file_name

wb = openpyxl.load_workbook(file_name)

new_sheet_name = "PagexSegment"
new_sheet = wb.create_sheet(title=new_sheet_name)

rows = dataframe_to_rows(df_GA4_TOTAL_PagexSegment, index=False, header=True)

for row in rows:
    new_sheet.append(row)

wb.save(file_name)